In [201]:
import numpy as np
from collections import defaultdict
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, IterableDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.tensorboard import SummaryWriter

import gymnasium as gym

In [220]:
a = [np.arange(3), np.arange(4)**2, np.arange(5)]
s = 0
for e in a: s+=len(e)
print(a)
print(s)

# np.ndarray.accumulate
np.sum(list(map(lambda e: len(e), a)))

[array([0, 1, 2]), array([0, 1, 4, 9]), array([0, 1, 2, 3, 4])]
12


12

In [94]:
x = torch.arange(10*3*1).reshape(5, 6)
t = torch.randint(0, 3, x.size()[:-1])

In [104]:
print('x', x)
print()
print('t', t)
print()
l = x[t == 0]
print('l', l)
l.mean(dim=0, dtype=float)
l.type(torch.float).mean(dim=0)
# idx = torch.tensor(idx, dtype=int).unsqueeze(0)
# torch.take_along_dim(x, idx, dim=0)

x tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29]])

t tensor([2, 1, 0, 1, 0])

l tensor([[12, 13, 14, 15, 16, 17],
        [24, 25, 26, 27, 28, 29]])


tensor([18., 19., 20., 21., 22., 23.])

In [38]:
t = torch.tensor([[10, 30, 20], [60, 40, 50]])
max_idx = torch.argmax(t)
torch.take_along_dim(t, max_idx)
sorted_idx = torch.argsort(t, dim=1)
torch.take_along_dim(t, sorted_idx, dim=1)

tensor([[10, 20, 30],
        [40, 50, 60]])

In [11]:
x = torch.rand(10, 4)
y = torch.zeros(10, 4)
print(f"error: {x - y}")
print(f"error^2: {(x - y)**2}")

F.mse_loss(x, y, reduction='none').size()

error: tensor([[0.9227, 0.1393, 0.3681, 0.6691],
        [0.6829, 0.2197, 0.8164, 0.6478],
        [0.7188, 0.5352, 0.4385, 0.5393],
        [0.1403, 0.3910, 0.8263, 0.3960],
        [0.5600, 0.0766, 0.3439, 0.3028],
        [0.7283, 0.7004, 0.3392, 0.9929],
        [0.7415, 0.8326, 0.9023, 0.8434],
        [0.6863, 0.9273, 0.9516, 0.4704],
        [0.2948, 0.5546, 0.1300, 0.7835],
        [0.5027, 0.6335, 0.4883, 0.5011]])
error^2: tensor([[0.8514, 0.0194, 0.1355, 0.4477],
        [0.4664, 0.0483, 0.6665, 0.4196],
        [0.5167, 0.2865, 0.1923, 0.2908],
        [0.0197, 0.1529, 0.6828, 0.1568],
        [0.3137, 0.0059, 0.1182, 0.0917],
        [0.5304, 0.4905, 0.1151, 0.9859],
        [0.5498, 0.6933, 0.8141, 0.7114],
        [0.4711, 0.8598, 0.9055, 0.2213],
        [0.0869, 0.3076, 0.0169, 0.6139],
        [0.2527, 0.4013, 0.2384, 0.2511]])


torch.Size([10, 4])

In [33]:
t = torch.tensor([[10, 30, 20], [60, 40, 50]])
max_idx = torch.argmax(t)
torch.take_along_dim(t, max_idx, )
sorted_idx = torch.argsort(t, dim=1)
torch.take_along_dim(t, sorted_idx, dim=1)



tensor(3)

In [22]:
t = torch.arange(10)
print(t.device)
print(t.to('cpu') is t)
print(t.device)

cpu
True
cpu


In [43]:
env = gym.make("Pendulum-v1", render_mode=None)#"human")
for e in dir(env):
    if e[0] != '_': print(e)

action_space
class_name
close
env
get_wrapper_attr
metadata
np_random
observation_space
render
render_mode
reset
reward_range
spec
step
unwrapped
wrapper_spec


In [40]:
observation, info = env.reset(seed=42)
observation

array([-0.14995256,  0.9886932 , -0.12224312], dtype=float32)

In [44]:
env = gym.make("Pendulum-v1")#, render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(1000):
    action = env.action_space.sample()  # this is where you would insert your policy
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        print(f"global_iter: {_}, terminated: {terminated}, truncated: {truncated}")
        observation, info = env.reset()

env.close()

global_iter: 199, terminated: False, truncated: True
global_iter: 399, terminated: False, truncated: True
global_iter: 599, terminated: False, truncated: True
global_iter: 799, terminated: False, truncated: True
global_iter: 999, terminated: False, truncated: True


### torch.utils.data.Dataset/DataLoader

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from dataset import CustomDataset, custom_collate_fn

# Dummy data (for demonstration purpose)
data = torch.randn(100, 2)  # 100 samples, each with 2 features

# Create an instance of your custom dataset
custom_dataset = CustomDataset(data)

# Create a DataLoader with dynamic trajectory sampling
batch_size = 10
shuffle = True
num_workers = 2
trajectory_sampling = True  # Set to True to sample trajectories, False for individual transitions

# Dynamically set trajectory sampling when creating DataLoader
dataloader = DataLoader(custom_dataset, 
                        batch_size=batch_size, 
                        shuffle=shuffle, 
                        num_workers=num_workers,
                        collate_fn=lambda batch: custom_collate_fn(batch, trajectory_sampling))

# Iterate over batches
for batch in dataloader:
    # Batch is a tuple containing input tensors
    inputs = batch
    if trajectory_sampling:
        print("Trajectory batch shape:", inputs.shape)  # shape of the batch when sampling trajectories
    else:
        print("Individual transition batch shape:", inputs.shape)  # shape of the batch when sampling individual transitions
    # Do whatever processing you need here


PicklingError: Can't pickle <function <lambda> at 0x1057f13a0>: attribute lookup <lambda> on __main__ failed